In [40]:
import pandas as pd 
import spacy
import bs4 
from bs4 import BeautifulSoup 
import urllib.request as urllib_request
from urllib.request import urlopen
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pickle
import time 

In [2]:
#Conexao com a base de dados
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.sprint3
materia_no_banco = db.materia

In [3]:
#Funcao para gerar a word cloud
def wordcloud_artigo(artigo):
    todas_palavras = ' '.join([texto for texto in artigo.texto_completo])
    nuvem_palvras = WordCloud(width= 500, height= 300, max_font_size = 110, collocations = False).generate(todas_palavras)
    plt.figure(figsize=(10,7))
    plt.imshow(nuvem_palvras, interpolation='bilinear')
    plt.title(f'{artigo.titulo[0]}')
    plt.axis("off")
    plt.show()  

#funcao para tirar espacos e caracteres nao alfabeticos
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)    
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [4]:
arquivo = open('link_das_materias.txt','r')
#Iterando cada link do arquivo para retirar os dados das materias
for link in arquivo:
    #Pegando o conteudo
    url = link
    response = urlopen(url)
    html = response.read()
    html = html.decode('utf-8')
        
    #Usando o BeautifulSoup para trabalhar com o conteudo
    soup = BeautifulSoup(html, 'html.parser')

    #Pegando titulo da materia
    titulo_materia = soup.find('h1', {"class": "c-content-head__title"}).get_text()
    titulo_materia = " ".join(titulo_materia.split())

    #Pegando os paragrafos da materia
    materia = soup.find('div', {'class': "c-news__body"}).findAll('p')
    paragrafos = []
    for paragrafo in materia:
        paragrafos.append(paragrafo.get_text())
    texto_completo = ' '.join(paragrafos)
        
    #Texto em minusculo
    paragrafos_minusculo = []
    for texto in paragrafos:
        texto = texto.lower()
        paragrafos_minusculo.append(texto)

    #Verificando se há as materias no banco e as inserindo se nao houver
    materia_para_o_banco = {
        'Titulo': titulo_materia,
        'Conteudo': paragrafos_minusculo
    }  

    #Reconhecendo no banco de dados
    materia_no_banco = db.Materias
    if not materia_no_banco.find_one({'Titulo': titulo_materia}):       
        insere = materia_no_banco.insert_one(materia_para_o_banco)
        print(f'Inserindo a materia: -| {titulo_materia}')
    else:
        print(f'Essa matéria já existe no Banco: -| {titulo_materia}')

    #Transformando o artigo em um data frame do pandas pra usar o spacy
    df_artigo = pd.DataFrame(index=[0] ,data={'titulo':titulo_materia, 'texto_completo': texto_completo})
    pd.set_option('display.max_colwidth', None)

    #usando spacy
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(str(df_artigo))

    #Lista NER
    entidades_nomeadas = nlp(str(df_artigo['texto_completo']))
    lista_de_entidades = []
    for token in entidades_nomeadas.ents:
        if token.orth_ not in lista_de_entidades and not token.orth_ == 'Name':
            lista_de_entidades.append(token.orth_)
        
    #tratando dados
    textos_normalizados = [trata_textos(doc) for doc in nlp.pipe(df_artigo['texto_completo'], batch_size= 500, n_process=-1)]

    #Verbos no artigo
    verbos_artigo = []
    for token in entidades_nomeadas:
        if token.pos_ == 'VERB' and token.is_alpha == True and token.orth_:
            verbos_artigo.append(token.orth_)

    #Prints
    print(f'Aqui estão os verbos utilizados nessa matéria: {verbos_artigo}')
    print(f'Aqui estão as entidades nomeadas dessa matéria: {lista_de_entidades}')
    print('------------------------------------------------')
    ########

    #Plotando a Word Cloud
    artigo = pd.DataFrame({
        'titulo': titulo_materia,
        'texto_completo': textos_normalizados
    },index=[0])    
    wordcloud_artigo(artigo)
arquivo.close()

Essa matéria já existe no Banco: -| 1971: Ofensiva de guerrilheiros no Camboja deixa 200 mortos
Aqui estão os verbos utilizados nessa matéria: ['conseguiram', 'destruir', 'estendeu', 'matou', 'destruiu', 'aumentou']
Aqui estão as entidades nomeadas dessa matéria: ['Guerrilheiros', 'Phnom Penh', 'Camboja', 'Indochina']
------------------------------------------------
Essa matéria já existe no Banco: -| 1971: Embaixador diz que, durante seu sequestro, lia notícias da negociação
Aqui estão os verbos utilizados nessa matéria: ['afirmou', 'acompanhava', 'raptado', 'aceitou', 'libertar', 'disse', 'sentiu', 'isolado', 'receber', 'pude', 'acompanhar', 'vigiavam', 'mascarados']
Aqui estão as entidades nomeadas dessa matéria: ['Suíça', 'Brasil', 'Giovanni Enrico Bucher', 'Chile', 'Bucher']
------------------------------------------------
Essa matéria já existe no Banco: -| 1971: Lista de presos que serão trocados por embaixador está completa
Aqui estão os verbos utilizados nessa matéria: ['infor

In [5]:
#Pegando os dados do csv e os separando em duas colunas
dataset = pd.read_csv('Tweets_Mg.csv')
classes = dataset['Classificacao'].values
tweets_bruto  = dataset["Text"].values

#texto para minusculo
textos = []
for texto in tweets_bruto:
    texto = texto.lower()
    textos.append(texto)

#tratando dados
textos_normalizados = [trata_textos(doc) for doc in nlp.pipe(textos, batch_size= 500, n_process=-1)]

#inserindo no dataset
dataset.insert(1,"texto_tratado",textos_normalizados)
dataset = dataset.drop_duplicates(subset='texto_tratado', keep='first')

In [6]:
#Definindo dataset separado
df_tweets = dataset[['texto_tratado','Classificacao']]

#tirando linhas duplicadas
df_tweets = df_tweets.drop_duplicates(subset='texto_tratado', keep='first')

#tirando linhas none
df_tweets = df_tweets.dropna()

In [7]:
#Criando nlp
nlp = spacy.load('pt_core_news_sm')
doc = nlp(str(df_tweets))

In [33]:
#Construindo o modelo
textos_modelo = df_tweets['texto_tratado'].values
classes_modelo = df_tweets['Classificacao'].values

vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(textos_modelo[:2700])

SEED = 22
np.random.seed(SEED)
modelo = MultinomialNB()
modelo.fit(freq_tweets, classes_modelo[:2700])


MultinomialNB()

In [34]:
testes = textos_modelo[2700:]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Negativo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positivo',
       'Positivo', 'Positivo', 'Positivo', 'Positivo', 'Positi

In [10]:
#Avaliando o modelo
resultados = cross_val_predict(modelo, freq_tweets, classes_modelo, cv = 10)

# acuracia
acuracia = metrics.accuracy_score(classes_modelo, resultados)
print(f'A acuracia foi de: {acuracia:.02f}% \n')

#medidas de avaliacao
print(metrics.classification_report(classes_modelo, resultados, labels=['Positivo', 'Negativo', 'Neutro']))

A acuracia foi de: 0.82% 

              precision    recall  f1-score   support

    Positivo       0.90      0.84      0.87      1422
    Negativo       0.65      0.89      0.75       556
      Neutro       0.85      0.78      0.81      1441

    accuracy                           0.82      3419
   macro avg       0.80      0.84      0.81      3419
weighted avg       0.84      0.82      0.83      3419



In [11]:
arquivo = open('link_das_materias.txt','r')
for link in arquivo:
    #conexão com o link
    url = link
    response = urlopen(url)
    html = response.read()
    html = html.decode('utf-8')
        
    #Usando o BeautifulSoup para trabalhar com o conteudo
    soup = BeautifulSoup(html, 'html.parser')

    #pegando titulo da materia
    titulo_materia = soup.find('h1', {"class": "c-content-head__title"}).get_text() 

    #Pegando os paragrafos da materia
    materia = soup.find('div', {'class': "c-news__body"}).findAll('p')

    print(f'E assim foram classificados os paragrafos da matéria: {titulo_materia}')
    for paragrafo in materia:
        freq_testes = vectorizer.transform([paragrafo.get_text()])
        previsao = modelo.predict(freq_testes)
        print(f'O paragrafo: {paragrafo.get_text()} -- Foi considerado: {previsao[0]}')
    print('--------------------------------------------------------------------------')
arquivo.close()

E assim foram classificados os paragrafos da matéria: 
    1971: Ofensiva de guerrilheiros no Camboja deixa 200 mortos
  
O paragrafo: Guerrilheiros cambojanos, vietcongues e norte-vietnamitas conseguiram destruir na madrugada desta sexta-feira (22) grande parte das instalações do aeroporto internacional de Phnom Penh, capital do Camboja. -- Foi considerado: Neutro
O paragrafo: Esse foi o primeiro ataque contra a cidade desde que a guerra na Indochina se estendeu ao Camboja, em março de 1970. A ação matou cerca de 200 pessoas, entre militares e civis.  -- Foi considerado: Positivo
O paragrafo: O fogo dos morteiros destruiu dois depósitos de munição, três hangares e vários aviões e helicópteros. Depois do bombardeio, um comando suicida com 18 homens aumentou a destruição. -- Foi considerado: Negativo
--------------------------------------------------------------------------
E assim foram classificados os paragrafos da matéria: 
    1971: Embaixador diz que, durante seu sequestro, lia no

In [29]:
dados_treino = df_tweets.to_dict('list')
valor_treino = ((dados_treino))


In [48]:
##Salvando o modelo no banco de dados
pickled_model = pickle.dumps(modelo)

#dados
dados_treino = df_tweets[:2700].to_dict('list')
valor_treino = ([dados_treino])
dados_teste = df_tweets[:2700].to_dict('list')
valor_teste = ([dados_teste])

#Conexao
client = MongoClient('mongodb://localhost:27017/')

#Banco
db = client.sprint3

#Colecao
modelo_no_banco = db['Modelo']

info = modelo_no_banco.insert_one({'MultinomialNB': pickled_model, 'dados_treino': dados_treino , 'dados_teste': dados_teste, 'created_time':time.time()})
print(info.inserted_id, 'Foi salvo com esse ID no banco!')
    
details = {
    'inserted_id':info.inserted_id,
    'MultinomialNB': pickled_model,
    'dados_treino':dados_treino,
    'dados_teste': dados_teste,
    'created_time':time.time()
}
print(details)

60bd8bbb18d4a8b9ea40bdc5 Foi salvo com esse ID no banco!
{'inserted_id': ObjectId('60bd8bbb18d4a8b9ea40bdc5'), 'MultinomialNB': b'\x80\x04\x95\xff\x01\x00\x00\x00\x00\x00\x00\x8c\x13sklearn.naive_bayes\x94\x8c\rMultinomialNB\x94\x93\x94)\x81\x94}\x94(\x8c\x05alpha\x94G?\xf0\x00\x00\x00\x00\x00\x00\x8c\tfit_prior\x94\x88\x8c\x0bclass_prior\x94N\x8c\x0en_features_in_\x94M9\x15\x8c\x0bn_features_\x94M9\x15\x8c\x08classes_\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x03\x85\x94h\x0e\x8c\x05dtype\x94\x93\x94\x8c\x02U8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNK K\x04K\x08t\x94b\x89C`N\x00\x00\x00e\x00\x00\x00g\x00\x00\x00a\x00\x00\x00t\x00\x00\x00i\x00\x00\x00v\x00\x00\x00o\x00\x00\x00N\x00\x00\x00e\x00\x00\x00u\x00\x00\x00t\x00\x00\x00r\x00\x00\x00o\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00P\x00\x00\x00o\x00\x00\x00s\x00\x00\x00i\x00\x00\x00t\x00\x00\x00i\x00\x00\x00v\x00\x